In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from label_to_cat import LABEL_TO_CAT

datapath = 'predict_probs_resnet50_short.h5'
BATCHES_COUNT = 12091

In [3]:
store = pd.HDFStore(datapath)
print('{} tables in storage'.format(len(store.keys())))

1 tables in storage


In [4]:
# assert len(store.keys()) == BATCHES_COUNT

preds = []
cur_id = None
cur_sum = None

for key in tqdm(store.keys()): 
    table = store.select(key)
    # table.drop(table[table.img_num != 0.].index, axis=0, inplace=True)
    assert table.shape[0] % 10 == 0
    for i in range(table.shape[0] / 10):
        product_id = table.pr_id.iloc[10*i]
        new_sum = table[10*i : 10*i + 10].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        
        if cur_id == product_id:
            assert cur_sum is not None
            cur_sum += new_sum
            continue
        elif cur_id is not None:
            assert cur_sum is not None
            id_ = int(cur_sum.idxmax()) - 1
            cat = LABEL_TO_CAT[id_]
            preds.append((int(cur_id), cat))
            
        cur_id = product_id
        cur_sum = new_sum
        
if cur_id is not None:
    id_ = int(cur_sum.idxmax()) - 1
    cat = LABEL_TO_CAT[id_]
    preds.append((int(cur_id), cat))
            
preds_df = pd.DataFrame(data=preds, columns=['_id','category_id'])
print('Dataframe shape', preds_df.shape)
preds_df.to_csv('preds_resnet50_all_imgs.csv', index=False)

100%|██████████| 1/1 [00:00<00:00,  1.41it/s]

('Dataframe shape', (254, 2))
